In [ ]:
%pip install sentence-transformers

# https://www.thepythoncode.com/article/finetune-bert-for-semantic-textual-similarity-in-python

In [2]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader, Dataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from transformers import BertTokenizer, BertModel, AdamW
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load your dataset into a Pandas DataFrame
df = pd.read_csv('mohler_dataset.csv')
# df = pd.read_csv('dataset.csv')

In [ ]:
df.shape

(249, 7)

In [ ]:
df.head()

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
0,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,High risk problems are address in the prototyp...,4.0,3.0,3.5
1,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To simulate portions of the desired final prod...,5.0,5.0,5.0
2,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,A prototype program simulates the behaviors of...,5.0,3.0,4.0
3,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...,5.0,5.0,5.0
4,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It is used to let the users have a first idea ...,3.0,3.0,3.0


In [ ]:
df.shape

(2273, 7)

In [ ]:
df['desired_answer'].dtype

dtype('O')

In [ ]:
# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert data into InputExample objects
train_examples = [InputExample(texts=[row['desired_answer'], row['student_answer']], label=row['score_avg']) for _, row in train_df.iterrows()]
test_examples = [InputExample(texts=[row['desired_answer'], row['student_answer']], label=row['score_avg']) for _, row in test_df.iterrows()]

# Create DataLoader for training and testing
train_dataloader = DataLoader(train_examples, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_examples, batch_size=16, shuffle=False)

In [ ]:
len(train_examples)

1818

## Using Sentence Tranformer

In [ ]:
# Load the pre-trained model
# model_name = 'paraphrase-MiniLM-L6-v2'
# sentence-transformers/all-mpnet-base-v2
# sentence-transformers/multi-qa-mpnet-base-dot-v1
# for best simeantic similarity : all-MiniLM-L6-v2
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Define the custom contrastive loss function
custom_loss = losses.ContrastiveLoss(model=model)

In [ ]:
# Fine-tune the model
model.fit(
    train_objectives=[(train_dataloader, custom_loss)],
    epochs=10,
    warmup_steps=100,
    evaluation_steps=500,
    output_path='fine_tuned_model/'
)

In [ ]:
len(test_examples)

455

In [ ]:
# Evaluate the model on the test data
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Define sentences1, sentences2, and scores for the evaluation
sentences1 = [example.texts[0] for example in test_examples]
sentences2 = [example.texts[1] for example in test_examples]
scores = [example.label for example in test_examples]

# Define the evaluation metric
evaluator = EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

In [ ]:
evaluator

In [ ]:
sentences1[:2]

['A walk around the tree, starting with the root, where each node is seen three times: from the left, from below, from the right.',
 'Keep the top of the stack toward the end of the array, so the push and pop operations will add or remove elements from the right side of the array.']

In [ ]:
sentences2[:3]

['it starts node on the left of the root and then proceeds to visits each node in a left to right order, visits the root, and then proceeds to repeat the previous step on the right side of the tree.',
 'Make an array, make the bottom at spot 0, make the last spot in the array the top, and increment top when you ad',
 'theta(n) the best case senario is that everything is already sorted']

In [ ]:
# Load the fine-tuned model from the output_path
fine_tuned_model = SentenceTransformer('fine_tuned_model/')

# Evaluate the model
model_score = evaluator(fine_tuned_model, output_path='fine_tuned_model/', epoch=10, steps=500)
print("Model score on test data:", model_score)

Model score on test data: 0.8660254037844387


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the fine-tuned model from the output_path
fine_tuned_model = SentenceTransformer('fine_tuned_model/')

text1 = "The quick brown fox jumps over the lazy dog."
text2 = "Gradient descent is an optimization algorithm"
# Encode the sentences to get sentence embeddings
embedding1 = fine_tuned_model.encode(text1, convert_to_tensor=True)
embedding2 = fine_tuned_model.encode(text2, convert_to_tensor=True)

# Calculate the cosine similarity score between the embeddings
similarity_score = util.pytorch_cos_sim(embedding1, embedding2)

print("Similarity score between sentence1 and sentence2:", similarity_score.item())


Similarity score between sentence1 and sentence2: -0.027629269286990166


In [ ]:
df.head(5)

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
0,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,High risk problems are address in the prototyp...,4.0,3.0,3.5
1,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To simulate portions of the desired final prod...,5.0,5.0,5.0
2,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,A prototype program simulates the behaviors of...,5.0,3.0,4.0
3,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...,5.0,5.0,5.0
4,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It is used to let the users have a first idea ...,3.0,3.0,3.0


## Fine tuning BERT

In [ ]:
# Not work perfectly...